# Registers generator

## General parameters

In [1]:
import math 
from datetime import datetime 
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [2]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_MV_WIDTH = 8                              # Input size for the block motion vectors (MV0 and MV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)

In [3]:
####################################################
# Rewrite registers
####################################################
# Template: [reg name, reg width, description, signed/unsigned]
REG_MATRIX = [["reg_coord", INPUT_COORD_WIDTH, 'Stores the coordenates', 'signed'],
              ["reg_MV", 2*INPUT_MV_WIDTH, 'Stores the input Motion Vectors. MSBs: horizontal, LSBs: vertical', 'signed'],
              ["reg_gen_MV", OUTPUT_MV_GEN_WIDTH, 'Stores the X and Y components of the MV generator output', 'signed'],
              ["reg_count_block", math.ceil(math.log2(INPUT_MACROBLOCK_SIZE)), 'Stores the counter to control which block will be processed', 'unsigned'],
              ["reg_input_line", INPUT_LINE_SIZE*PIXEL_WIDTH, 'Stores the integer input samples of pixels', 'unsigned'],
              ["reg_counter", 4, 'Stores the counter value to control all the loops', 'unsigned'],
              ["reg_int_out", OUT_2ND_INTERP_WIDTH, 'Stores the interpolator output line', 'signed'],
              ["reg_internal_buffer", OUT_1ST_INTERP_WIDTH, 'Stores one sample. It is used to build the transposed buffer.', 'signed'], 
              ['reg_buf_col_counter', 2, 'Stores the mux_sel_bffer_col', '']
              ]

In [4]:
###############################################
# Register text template
###############################################
def generate_reg(ind):    
    # Modified date
    DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))
    # Register file text
    register_verilog = '''/*-----------------------------------------------------------------------------------
* File: '''+REG_MATRIX[ind][0]+'''.v
* Date generated: 25/03/2023
* Date modified: '''+DATE_MODIF+'''
* Author: Bruna Suemi Nagai
* Description: '''+str(REG_MATRIX[ind][2])+'''
*----------------------------------------------------------------------------------- */

module '''+REG_MATRIX[ind][0]+''' (
    CLK,
    RST_ASYNC_N, 
    WRITE_EN,
    DATA_IN,  
    DATA_OUT
);


// ------------------------------------------
// IO declaration
// ------------------------------------------
    input CLK;                              // Clock
    input RST_ASYNC_N;						// Asynchronous reset
    input WRITE_EN;							// Enables writing
    input '''+REG_MATRIX[ind][3]+''' ['''+str(REG_MATRIX[ind][1]-1)+''':0] DATA_IN;			// Data in
    output reg '''+REG_MATRIX[ind][3]+''' ['''+str(REG_MATRIX[ind][1]-1)+''':0] DATA_OUT;	    // Data out
    

// ------------------------------------------
// Sequential logic
// ------------------------------------------
always @(posedge CLK, negedge RST_ASYNC_N) begin
if (!RST_ASYNC_N)                        // If rst async is low
    begin
            DATA_OUT <= '''+str(REG_MATRIX[ind][1])+''''b0;
    end
    
    else if (WRITE_EN) 		 			    // If write enable is high
    begin
        DATA_OUT <= DATA_IN; 			    // Write data to the register at the specified address
    end 
end

endmodule // '''+REG_MATRIX[ind][0]+'''
    '''
    # Write the register file
    with open('../verilog/registers/'+REG_MATRIX[ind][0]+'.v', 'w') as reg_file:
        reg_file.write(register_verilog)


In [5]:
###############################################
# Simulation.do text template
###############################################
def generate_simu(ind):
    SIMU_0 =[]
    SIMU_1 = []
    SIMU_MAX_POS = []
    SIMU_MAX_NEG = []
    for a in range(REG_MATRIX[ind][1]):
        SIMU_0.append('0')

        if a <= REG_MATRIX[ind][1]-2:
            SIMU_1.append('0')
        else:
            SIMU_1.append('1')
        
        if a == 0:
            SIMU_MAX_POS.append('0')
            SIMU_MAX_NEG.append('1')
        else:
            SIMU_MAX_POS.append('1')
            SIMU_MAX_NEG.append('0')
    SIMU_0_concat = ''.join(SIMU_0)
    SIMU_1_concat = ''.join(SIMU_1)
    SIMU_MAX_POS_concat = ''.join(SIMU_MAX_POS)
    SIMU_MAX_NEG_concat = ''.join(SIMU_MAX_NEG)

    simu_do = '''restart -f

force -freeze sim:/'''+REG_MATRIX[ind][0]+'''/CLK 1 0, 0 {5 ns} -r {10 ns}
force -freeze sim:/'''+REG_MATRIX[ind][0]+'''/RST_ASYNC_N 0 0 ns, 1 5 ns
force -freeze sim:/'''+REG_MATRIX[ind][0]+'''/WRITE_EN 0 0 ns, 1 28 ns, 0 32 ns, 1 38 ns, 0 42 ns, 1 48 ns, 0 52 ns
force -freeze sim:/'''+REG_MATRIX[ind][0]+'''/DATA_IN '''+SIMU_0_concat+''' 15 ns 
force -freeze sim:/'''+REG_MATRIX[ind][0]+'''/DATA_IN '''+SIMU_1_concat+''' 25 ns
force -freeze sim:/'''+REG_MATRIX[ind][0]+'''/DATA_IN '''+SIMU_MAX_POS_concat+''' 35 ns
force -freeze sim:/'''+REG_MATRIX[ind][0]+'''/DATA_IN '''+SIMU_MAX_NEG_concat+''' 45 ns

run 60 ns '''
    # Write the simulation file
    with open('../verilog/registers/'+REG_MATRIX[ind][0][4:]+'_simu.do', 'w') as simu_file:
        simu_file.write(simu_do)

In [7]:
# Define the list of words to select from
reg_names = ['Select a register', 'reg_coord', 'reg_MV', 'reg_gen_MV', 'reg_count_block', 
             'reg_input_line', 'reg_counter', 'reg_int_out', 'reg_internal_buffer', 'reg_buf_col_counter', 'Generate all']

# Define the initial values 
SELECTED_REG = 'REG_NOOB'
INDEX = 0

# Create the dropdown widget
word_select = widgets.Dropdown(
    options=reg_names,
    value=reg_names[0],
    description='Register:')

# Create the button widget
button = widgets.Button(description='Generate register file')

# Define a function to update the FILE_NAME variable
def update_filename(b):
    global INDEX
    SELECTED_REG = word_select.value
    if SELECTED_REG == 'reg_coord':
        INDEX = 0
    elif SELECTED_REG == 'reg_MV':
        INDEX = 1
    elif SELECTED_REG == 'reg_gen_MV':
        INDEX = 2
    elif SELECTED_REG == 'reg_count_block':
        INDEX = 3
    elif SELECTED_REG == 'reg_input_line':
        INDEX = 4
    elif SELECTED_REG == 'reg_counter':
        INDEX = 5
    elif SELECTED_REG == 'reg_int_out':
        INDEX = 6
    elif SELECTED_REG == 'reg_internal_buffer':
        INDEX = 7
    elif SELECTED_REG == 'reg_buf_col_counter':
        INDEX = 8
    elif SELECTED_REG == 'Generate all':
        INDEX = 9
    else:
        INDEX = 10

    # Loop for generate all    
    if INDEX == 9:
        for i in range(INDEX):
            generate_reg(i)
            generate_simu(i)
            print('Register ', REG_MATRIX[i][0], ' generated.')
            print('Index =', i)
        print('Finished generating all registers :)')
    else:
        generate_reg(INDEX)
        generate_simu(INDEX)
        print('Register ', SELECTED_REG, ' generated.')
        print('Index =', INDEX)

# Attach the function to the button's on-click event
button.on_click(update_filename)

# Display the widgets
display(word_select)
display(button)


Dropdown(description='Register:', options=('Select a register', 'reg_coord', 'reg_MV', 'reg_gen_MV', 'reg_coun…

Button(description='Generate register file', style=ButtonStyle())